In [15]:
!pip3 install feedparser

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


In [3]:
import feedparser
import pandas as pd
from datetime import datetime
import time
import psycopg2
import psycopg2.extras
from hashlib import md5
from base64 import b64encode

In [204]:
conn = psycopg2.connect(
    host="dpg-ckho6v6afg7c73fit3qg-a.singapore-postgres.render.com",
    database="dev_performo",
    user="performo",
    password="mLDsC1aDIccyzYdDQNqmU9xTnPfifxRy")
cursor = conn.cursor()
cursor.execute("""select 
        publisher_salt,id, feed_url 
        from dev_performo.publisher_category_mapping pcm 
        where publisher_id = 6
        LIMIT 4
        """)
data = cursor.fetchall()

In [205]:
data

[('ITO-en-in', 400, 'https://www.indiatoday.in/rss/1206509'),
 ('ITO-en-in', 396, 'https://www.indiatoday.in/rss/home'),
 ('ITO-en-in', 397, 'https://www.indiatoday.in/rss/1206514'),
 ('ITO-en-in', 398, 'https://www.indiatoday.in/rss/1206551')]

In [206]:
df = pd.DataFrame(columns=['id', 'title', 'pubdate','link', 'pub_category_id', 'author','guid', 'summary', 'mediaurl'])
count=0
for pub_id,idx,url in data:
    feed = feedparser.parse(url)
    for post in feed["entries"]:
        print(post.keys())
        print(post)
        break
#         exec(structs["News18"]) 
#         df = pd.concat([df,dat],ignore_index=True)
        try:
            df = pd.concat(
                [   df,
                    pd.DataFrame(
                        [
                            [
                                str(pub_id)+"-"+b64encode(md5(post["id"].encode('utf-8')).digest()).decode("utf-8"),#publisherID+Hash of GUID
                                post["title"], 
                                datetime.fromtimestamp(time.mktime(post["published_parsed"])).isoformat(),
                                post["link"],
                                idx,
                                None,
                                post["id"],
                                post["summary"],
                                post["media_content"][0]["url"]
                                
                            ]
                        ],
                            columns=df.columns)
                ],
                ignore_index=True
        )
        except:
            count = count+1
print(count)

dict_keys(['title', 'title_detail', 'links', 'link', 'summary', 'summary_detail', 'published', 'published_parsed'])
{'title': 'Assembly polls 2023 | Battling the odds', 'title_detail': {'type': 'text/plain', 'language': None, 'base': 'https://www.indiatoday.in/rss/1206509', 'value': 'Assembly polls 2023 | Battling the odds'}, 'links': [{'rel': 'alternate', 'type': 'text/html', 'href': 'https://www.indiatoday.in/magazine/cover-story/story/20231023-assembly-polls-2023-battling-the-odds-2448338-2023-10-13?utm_source=rss'}], 'link': 'https://www.indiatoday.in/magazine/cover-story/story/20231023-assembly-polls-2023-battling-the-odds-2448338-2023-10-13?utm_source=rss', 'summary': '<a href="https://www.indiatoday.in/magazine/cover-story/story/20231023-assembly-polls-2023-battling-the-odds-2448338-2023-10-13"> <img align="left" alt="" border="0" height="180" hspace="2" src="https://akm-img-a-in.tosshub.com/indiatoday/images/story/202310/battling-the-odds-1355512-16x9.jpg?VersionId=o.YB2XKB9mJZ

In [170]:
df.id.nunique()
# df = pd.concat([df.head(),df.tail()],ignore_index=True)
# df

91

In [171]:
df.shape

(118, 9)

In [110]:
# cur.execute("PREPARE stmt")
psycopg2.extras.execute_batch(
    cursor, 
    "INSERT INTO {} ({}) {} ON CONFLICT(id) DO UPDATE SET {}".format(
        "dev_performo.article_master",
        ",".join(df.columns),
        "VALUES({})".format(",".join(["%s" for _ in df.columns])),
        ",".join([i+"=EXCLUDED."+i for i in df.columns])), 
    df.values
)
# cur.execute("DEALLOCATE stmt")
conn.commit()
conn.close()

In [114]:
"UPDATE {} SET {} FROM {} AS t({}) WHERE am.id=am.id AND am.pubdate<>t.pubdate".format("dev_performo.article_master am",",".join([i+"=t."+i for i in df.columns]),"VALUES({})".format(",".join(["%s" for _ in df.columns])),",".join(df.columns)) 

'UPDATE dev_performo.article_master am SET id=t.id,title=t.title,pubdate=t.pubdate,link=t.link,pub_category_id=t.pub_category_id,author=t.author,guid=t.guid,summary=t.summary,mediaurl=t.mediaurl FROM VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s) AS t(id,title,pubdate,link,pub_category_id,author,guid,summary,mediaurl) WHERE am.id=am.id AND am.pubdate<>t.pubdate'

In [112]:
"UPDATE dev_performo.article_master am SET id=t.id,title=t.title,pubdate=t.pubdate,link=t.link,pub_category_id=t.pub_category_id,author=t.author,guid=t.guid,summary=t.summary,mediaurl=t.mediaurl FROM (values('test','updated test Title',NOW(),'test',2,'test','test','test','test')) as t(id,title,pubdate,link,pub_category_id,author,guid,summary,mediaurl) WHERE am.id=t.id AND am.pubdate<>t.pubdate


"UPDATE dev_performo.article_master am SET id=t.id,title=t.title,pubdate=t.pubdate,link=t.link,pub_category_id=t.pub_category_id,author=t.author,guid=t.guid,summary=t.summary,mediaurl=t.mediaurl FROM (values('test','updated test Title',NOW(),'test',2,'test','test','test','test')) as t(id,title,pubdate,link,pub_category_id,author,guid,summary,mediaurl) WHERE am.id=t.id AND am.pubdate<>t.pubdate"

In [159]:
",".join(["table."+i+"=EXCLUDED."+i for i in df.columns])

'table.id=EXCLUDED.id{},table.title=EXCLUDED.title{},table.pubdate=EXCLUDED.pubdate{},table.link=EXCLUDED.link{},table.pub_category_id=EXCLUDED.pub_category_id{},table.author=EXCLUDED.author{},table.guid=EXCLUDED.guid{},table.summary=EXCLUDED.summary{},table.mediaurl=EXCLUDED.mediaurl'

INDIAN EXPRESS = Not accessible data<br>
HT = Data not present for some categories<br>
Some posts are repetitive in feed<br>


In [201]:
structs = {"NWS18":"""dat = pd.DataFrame(
                        [
                            [
                                str(pub_id)+"-"+b64encode(md5(post["id"].encode('utf-8')).digest()).decode("utf-8"),#publisherID+Hash of GUID
                                post["title"],
                                dt,
                                post["link"],
                                idx,
                                None,
                                post["id"],
                                post["summary"],
                                post["media_content"][0]["url"]
                                
                            ]
                        ],
                            columns=df.columns)""",
           "CNBC":"""dat = pd.DataFrame(
                        [
                            [
                                str(pub_id)+"-"+b64encode(md5(post["link"].encode('utf-8')).digest()).decode("utf-8"),#publisherID+Hash of GUID
                                post["title"],
                                dt,
                                post["link"],
                                idx,
                                None,
                                post["id"],
                                post["summary"],
                                None
                                
                            ]
                        ],
                            columns=df.columns)""",
           "DAJ":"""dat = pd.DataFrame(
                        [
                            [
                                str(pub_id)+"-"+b64encode(md5(post["id"].encode('utf-8')).digest()).decode("utf-8"),#publisherID+Hash of GUID
                                post["title"],
                                dt,
                                post["link"],
                                idx,
                                None,
                                post["id"],
                                post["summary"],
                                post["media_content"][0]["url"]
                                
                            ]
                        ],
                            columns=df.columns)""",
           "FP":"""dat=pd.DataFrame(
                        [
                            [
                                str(pub_id)+"-"+b64encode(md5(post["id"].encode('utf-8')).digest()).decode("utf-8"),#publisherID+Hash of GUID
                                post["title"],
                                datetime.fromtimestamp(time.mktime(post["published_parsed"])).isoformat(),
                                post["link"],
                                idx,
                                post['author'],
                                post["id"],
                                post["summary"],
                                post["media_content"][0]["url"]
                                
                            ]
                        ],
                            columns=df.columns)""",
           "HT":"""dat=pd.DataFrame(
                        [
                            [
                                str(pub_id)+"-"+b64encode(md5(post["id"].encode('utf-8')).digest()).decode("utf-8"),#publisherID+Hash of GUID
                                post["title"],
                                datetime.fromtimestamp(time.mktime(post["published_parsed"])).isoformat(),
                                post["link"],
                                idx,
                                None,
                                post["id"],
                                post["summary"],
                                post["media_content"][0]["url"]
                                
                            ]
                        ],
                            columns=df.columns)"""
           
          }

In [202]:
exec(structs["HT"])

In [203]:
dat

,id,title,pubdate,link,pub_category_id,author,guid,summary,mediaurl
0,HT-en-in-rs18VnYlGgHBtyLpNEzNZA==,Weekly Numerology Predictions from 16th to 22n...,2023-10-16T06:35:30,https://www.hindustantimes.com/astrology/horos...,170,None,https://www.hindustantimes.com/astrology/horos...,Bejan Daruwala's son Chirag Daruwala writes we...,https://www.hindustantimes.com/ht-img/img/2023...
